In [2]:
import os
import pandas as pd

In [22]:
filenames = next(os.walk("Исходники"), (None, None, []))[2]

In [24]:
for file in filenames:
    name = file.split("_")[1]

    new_name = ''
    for char in range(len(name)):
        if char == 2:
            new_name += '.' + name[char]
        elif char == 4:
            new_name += '.' + '20' + name[char]
        else:
            new_name += name[char]
    os.rename(f'Исходники/{file}', f'Исходники/{new_name}')

In [3]:
import tabula
import camelot
from tqdm import tqdm

In [4]:
result = pd.DataFrame()
page = True
filenames = next(os.walk("Исходники"), (None, None, []))[2]
broken = {}
broken_table = []

for file in tqdm(filenames[453:]):
    #Была ошибка на данном этапе - исправлена. Для воспроизведения результата срез надо убрать
    for page in range(2,1_000): 
        try:
            pdf_df = tabula.read_pdf(f"Исходники/{file}", pages = f'{page}', multiple_tables = True, lattice=True)
        except:
            break
        
        for table in pdf_df:
            table = table.dropna(axis=1, how='all')
            if len(table.columns) < 6:
                continue
            if len(table) >3:
                if file[:-4] in list(broken.keys()):
                    broken[file[:-4]] += [page]
                else:
                    broken[file[:-4]] = [page]
                continue

            for i in range(len(table.columns)):
                breaks = False
                if not pd.isna(table.iat[0,i]):
                    col = table.iat[0,i].split("\r")
                    
                    if "НКД, руб" in col and len(table.columns) != 16:
                        table = table.iloc[:, :-1]
                        break

            if len(table) > 2 and not pd.isna(table.iat[2,1]):
                emitent = table.iat[2,1].split("\r")
                type = table.iat[2,2].split("\r")
                if type[0] == "иф":
                    #Иностранные акции и фонды в рассмотрение не берем
                    break

                close = str(table.iat[2,-7]).replace(",", "").replace(" ", "").replace("-", "0.0").split("\r")  
                value = str(table.iat[2,-3]).replace(",", "").replace(" ", "").replace("-", "0.0").split("\r")
                volume = str(table.iat[2,-2]).replace(",", "").replace(" ", "").replace("-", "0.0").split("\r")
                number = str(table.iat[2,-1]).replace(",", "").replace(" ", "").replace("-", "0.0").split("\r")
                if len(emitent) == len(type) == len(close) == len(close) == len(value) == len(volume) == len(number):
                    # Периодически они вместо нулевых значений ставят не 0 и не - а просто пропуск. С такими файлами - отдельная работа
                    day_df = pd.DataFrame({"Дата": [file[:-4]]*len(type),
                                            "Эмитент": emitent,
                                            "Тип": type,
                                            "Цена закрытия": close,
                                            "Объем сделок, руб": value,
                                            "Объем сделок, шт": volume,
                                            "Кол-во сделок": number,
                                            })
                    result = pd.concat([result,day_df], ignore_index=True)   
                    broken_table += [file]                     
        if type[0] == "иф":
                break

  0%|          | 1/1987 [00:29<16:29:44, 29.90s/it]Error from tabula-java:
Exception in thread "main" java.lang.IndexOutOfBoundsException: Page number does not exist.
	at technology.tabula.ObjectExtractor.extractPage(ObjectExtractor.java:19)
	at technology.tabula.PageIterator.next(PageIterator.java:30)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:161)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)


  0%|          | 8/1987 [02:25<13:00:48, 23.67s/it]Error from tabula-java:
Exception in thread "main" java.lang.IndexOutOfBoundsException: Page number does not exist.
	at technology.tabula.ObjectExtractor.extractPage(ObjectExtractor.java:19)
	at technology.tabula.PageIterator.next(PageIterator.java:30)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:161)
	at technology.tabul

In [5]:
len(result)

2086882

In [6]:
result.head()

,Дата,Эмитент,Тип,Цена закрытия,"Объем сделок, руб","Объем сделок, шт",Кол-во сделок
0,06.11.2020,+МосЭнерго,ао,1.9890,25590961.50,12700000,1004
1,06.11.2020,Petropavl,ао,29.200,235623524.50,8116500,7486
2,06.11.2020,Polymetal,ао,1926.7,2774830491.60,1459767,30593
3,06.11.2020,Yandex clA,ао,4904.8,4274181858.20,882470,33959
4,06.11.2020,Акрон,ао,6348,23625996.00,3698,980


In [7]:
result['Тип'].unique()

array(['ао', 'ап', 'об', 'ип', 'ис', 'др'], dtype=object)

In [11]:
len(set(broken_table))

1932

In [13]:
len(broken)

362

In [43]:
total_result = result.copy()

In [14]:
prev_result = pd.read_excel("Результат.xlsx")

In [44]:
total_result = pd.concat([total_result,prev_result], ignore_index=True)

In [45]:
total_result = total_result.iloc[:,:-1]

In [46]:
total_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566347 entries, 0 to 2566346
Data columns (total 7 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Дата               object
 1   Эмитент            object
 2   Тип                object
 3   Цена закрытия      object
 4   Объем сделок, руб  object
 5   Объем сделок, шт   object
 6   Кол-во сделок      object
dtypes: object(7)
memory usage: 137.1+ MB


In [47]:
total_result['Дата'] = pd.to_datetime(total_result['Дата'],dayfirst = True)

In [48]:
total_result['Цена закрытия'] = pd.to_numeric(total_result['Цена закрытия'], errors = "coerce")
total_result['Объем сделок, руб'] = pd.to_numeric(total_result['Объем сделок, руб'], errors = "coerce")
total_result['Объем сделок, шт'] = pd.to_numeric(total_result['Объем сделок, шт'], errors = "coerce")
total_result['Кол-во сделок'] = pd.to_numeric(total_result['Кол-во сделок'], errors = "coerce")

In [50]:
total_result = total_result.sort_values(by = ['Дата','Тип','Эмитент','Цена закрытия'])

In [71]:
for year in range(2013, 2022+1):
    total_result.drop(total_result[pd.DatetimeIndex(total_result['Дата']).year != year].index).to_excel(f'Ежегодные данные/{year}.xlsx')

In [57]:
total_result

,Дата,Эмитент,Тип,Цена закрытия,"Объем сделок, руб","Объем сделок, шт",Кол-во сделок
1544750,2013-03-25,ГАЗПРОМ ао,ао,138.58,12746541.30,90360.0,71.0
1544749,2013-03-25,МосБиржа,ао,51.83,1961289.00,37600.0,29.0
1544751,2013-03-25,ОФЗ 25068,об,108.38,0.00,0.0,0.0
1544752,2013-03-25,ОФЗ 25071,об,103.50,0.00,0.0,0.0
1544753,2013-03-25,ОФЗ 25075,об,102.02,0.00,0.0,0.0
...,...,...,...,...,...,...,...
1940128,2022-08-29,ЯрОбл35017,об,99.84,0.00,0.0,0.0
1940131,2022-08-29,ЯрОбл35018,об,0.00,0.00,0.0,0.0
1940130,2022-08-29,ЯрОбл35018,об,96.02,25324.58,35.0,16.0
1942139,2022-08-29,реСторБ1Р1,об,0.00,0.00,0.0,0.0
